In [1]:
import pandas as pd
# !pip install gensim==3.8.3

In [2]:
df_fullinfo = pd.read_csv('df_fullinfo.csv', sep=';')
df_fullinfo = df_fullinfo.set_index('imdb_id')
df_fullinfo = df_fullinfo.loc[:,['moviename', 'subs_text_proc']]
df_subs = pd.read_csv('df_subs.csv', sep=';', index_col=0)
df_subs = df_subs.set_index('imdb_id')
df_subs=df_subs.loc[:,['subs_text']]
df = df_subs.join(df_fullinfo)
df['subs_len'] = df['subs_text'].apply(lambda x: len(x.split()))
df['subs_proc_len'] = df['subs_text_proc'].apply(lambda x: len(x.split()))
films_to_handy = [75686,109830,76759,119116,120586,118715,88763,110413,120735,24216]
for imdb_id in films_to_handy:
    moviename = df[df.index == imdb_id].moviename.item()
    subslen = df[df.index == imdb_id].subs_len.item()
    subs_proc_len = df[df.index == imdb_id].subs_proc_len.item()
    print(f'{moviename}, subs_len: {subslen}, subs_proc_len: {subs_proc_len}')
df_handy = df[df.index.isin(films_to_handy)]
df_handy['subs_text_proc'] = df_handy['subs_text_proc'].apply(eval)
df_handy

Annie Hall, subs_len: 13351, subs_proc_len: 6304
Forrest Gump, subs_len: 11440, subs_proc_len: 5729
Star Wars: Episode IV - A New Hope, subs_len: 10093, subs_proc_len: 4779
The Fifth Element, subs_len: 8053, subs_proc_len: 3969
American History X, subs_len: 12080, subs_proc_len: 5841
The Big Lebowski, subs_len: 10796, subs_proc_len: 5280
Back to the Future, subs_len: 9324, subs_proc_len: 4716
Leon: The Professional, subs_len: 6841, subs_proc_len: 3302
Lock, Stock and Two Smoking Barrels, subs_len: 10353, subs_proc_len: 4891
King Kong, subs_len: 6160, subs_proc_len: 2965


/tmp/ipykernel_8886/3389496019.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_handy['subs_text_proc'] = df_handy['subs_text_proc'].apply(eval)


,subs_text,moviename,subs_text_proc,subs_len,subs_proc_len
imdb_id,,,,,
76759,Did you hear that?\nThey shut down the main re...,Star Wars: Episode IV - A New Hope,"[hear, shut, main, reactor, destroy, sure, mad...",10093,4779
109830,Hello. My name's\nForrest... Forrest Gump.\nDo...,Forrest Gump,"[hello, name, forrest, forrest, gump, want, ch...",11440,5729
119116,"Come, come!\nPlease!\nAziz. Aziz.\nAziz, light...",The Fifth Element,"[come, come, please, aziz, aziz, aziz, light, ...",8053,3969
120586,"Derek.\nDerek.\nFucking pervert, Dan.\n- Jesus...",American History X,"[derek, derek, fuck, pervert, dan, jesus, fuck...",12080,5841
120735,"Right, let's sort the buyers\nfrom the spyers,...","Lock, Stock and Two Smoking Barrels","[right, let, sort, buyers, spyers, needy, gree...",10353,4891
110413,"- Allora, come stai, Leone?\n- Bene\nOkay, oka...",Leon: The Professional,"[allora, come, stai, leone, bene, okay, okay, ...",6841,3302
88763,"October is\ninventory time,\nso right now,\nSt...",Back to the Future,"[october, inventory, time, right, statler, toy...",9324,4716
118715,"Out west there was this fella,\nfella I want t...",The Big Lebowski,"[west, fella, fella, want, tell, fella, name, ...",10796,5280
24216,"Say, is this the moving-picture ship?\nThe Ven...",King Kong,"[say, movingpicture, ship, venture, yeah, go, ...",6160,2965


In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
# Gensim
# !pip install gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy
# Plotting tools
# !pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

2023-05-15 06:09:50.489842: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-15 06:09:50.531974: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-15 06:09:50.532554: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 06:09:51.255872: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-15 06:09:51.889543: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [4]:
data_words = df_handy.subs_text_proc
# data_words = df.subs_text_proc


In [5]:
# !pip install nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence)))  # deacc=True removes punctuations

data_words = list(sent_to_words(data_words))

print(data_words[:1])

[['hear', 'shut', 'main', 'reactor', 'destroy', 'sure', 'madness', 'doom', 'escape', 'princess', 'time', 'whats', 'rd', 'last', 'head', 'direction', 'go', 'send', 'kessel', 'spice', 'mine', 'smash', 'know', 'wait', 'minute', 'go', 'death', 'star', 'plan', 'main', 'computer', 'transmissions', 'intercept', 'do', 'plan', 'intercept', 'transmissions', 'consular', 'ship', 'diplomatic', 'mission', 'consular', 'ship', 'ambassador', 'commander', 'tear', 'ship', 'apart', 'find', 'plan', 'bring', 'passengers', 'want', 'alive', 'theres', 'one', 'set', 'stun', 'right', 'inform', 'lord', 'vader', 'prisoner', 'hey', 'permit', 'restrict', 'deactivate', 'sure', 'call', 'mindless', 'philosopher', 'overweight', 'glob', 'grease', 'come', 'somebody', 'see', 'secret', 'mission', 'plan', 'talk', 'im', 'get', 'im', 'go', 'regret', 'go', 'another', 'one', 'hold', 'fire', 'theres', 'life', 'form', 'must', 'shortcircuited', 'thats', 'funny', 'damage', 'look', 'bad', 'sure', 'thing', 'safe', 'darth', 'vader', 'c

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=3, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['hear', 'shut', 'main', 'reactor', 'destroy', 'sure', 'madness', 'doom', 'escape', 'princess', 'time', 'whats', 'rd', 'last', 'head', 'direction', 'go', 'send', 'kessel', 'spice', 'mine', 'smash', 'know', 'wait_minute', 'go', 'death_star', 'plan', 'main', 'computer', 'transmissions', 'intercept', 'do', 'plan', 'intercept', 'transmissions', 'consular', 'ship', 'diplomatic', 'mission', 'consular', 'ship', 'ambassador', 'commander', 'tear', 'ship', 'apart', 'find', 'plan', 'bring', 'passengers', 'want', 'alive', 'theres', 'one', 'set', 'stun', 'right', 'inform', 'lord_vader', 'prisoner', 'hey', 'permit', 'restrict', 'deactivate', 'sure', 'call', 'mindless', 'philosopher', 'overweight', 'glob', 'grease', 'come', 'somebody', 'see', 'secret', 'mission', 'plan', 'talk', 'im', 'get', 'im', 'go', 'regret', 'go', 'another', 'one', 'hold', 'fire', 'theres', 'life', 'form', 'must', 'shortcircuited', 'thats', 'funny', 'damage', 'look', 'bad', 'sure', 'thing', 'safe', 'darth_vader', 'could', 'bold'

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_words_bigrams[:1])

[['hear', 'shut', 'main', 'reactor', 'destroy', 'sure', 'madness', 'doom', 'escape', 'princess', 'time', 'whats', 'rd', 'last', 'head', 'direction', 'go', 'send', 'kessel', 'spice', 'mine', 'smash', 'know', 'wait_minute', 'go', 'death_star', 'plan', 'main', 'computer', 'transmissions', 'intercept', 'do', 'plan', 'intercept', 'transmissions', 'consular', 'ship', 'diplomatic', 'mission', 'consular', 'ship', 'ambassador', 'commander', 'tear', 'ship', 'apart', 'find', 'plan', 'bring', 'passengers', 'want', 'alive', 'theres', 'one', 'set', 'stun', 'right', 'inform', 'lord_vader', 'prisoner', 'hey', 'permit', 'restrict', 'deactivate', 'sure', 'call', 'mindless', 'philosopher', 'overweight', 'glob', 'grease', 'come', 'somebody', 'see', 'secret', 'mission', 'plan', 'talk', 'im', 'get', 'im', 'go', 'regret', 'go', 'another', 'one', 'hold', 'fire', 'theres', 'life', 'form', 'must', 'shortcircuited', 'thats', 'funny', 'damage', 'look', 'bad', 'sure', 'thing', 'safe', 'darth_vader', 'could', 'bold

In [9]:
# data_words_bigrams.save('fil-topicmod-bigram_mod_full.pkl')
# bag_of_words = [data_words_bigrams[w] for w in tokens]
# with open('bigrams.pkl', 'wb') as f:
#     pickle.dump(bag_of_words, f)

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 6), (4, 3), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 2), (13, 1), (14, 1), (15, 1), (16, 5), (17, 1), (18, 1), (19, 1), (20, 2), (21, 6), (22, 3), (23, 2), (24, 2), (25, 3), (26, 1), (27, 6), (28, 2), (29, 1), (30, 2), (31, 2), (32, 1), (33, 2), (34, 4), (35, 2), (36, 1), (37, 2), (38, 1), (39, 1), (40, 2), (41, 2), (42, 1), (43, 3), (44, 2), (45, 3), (46, 2), (47, 1), (48, 4), (49, 2), (50, 1), (51, 3), (52, 4), (53, 1), (54, 1), (55, 1), (56, 1), (57, 10), (58, 2), (59, 2), (60, 9), (61, 1), (62, 1), (63, 1), (64, 1), (65, 22), (66, 11), (67, 1), (68, 1), (69, 2), (70, 4), (71, 1), (72, 1), (73, 2), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 4), (80, 2), (81, 1), (82, 4), (83, 1), (84, 2), (85, 5), (86, 4), (87, 2), (88, 2), (89, 1), (90, 8), (91, 1), (92, 2), (93, 1), (94, 1), (95, 4), (96, 15), (97, 3), (98, 4), (99, 2), (100, 1), (101, 5), (102, 1), (103, 2), (104, 2), (105, 1), (106, 2), (107, 2), (108, 3), (109, 1), (110

## Построение тематической модели

In [11]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

## Просмотр тем в модели LDA

In [12]:
# lda_model.print_topics()

In [13]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.0934912245434

Coherence Score:  0.23413836763524803


## Визуализация темы и ключевых слов

In [14]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')
pyLDAvis.show(vis, local=False)

Serving to http://127.0.0.1:8888/    [Ctrl-C to exit]


/home/eva/.venvs/kerasenv/lib/python3.8/site-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(
127.0.0.1 - - [15/May/2023 06:10:05] "GET / HTTP/1.1" 200 -



stopping Server...


## LDA модель Mallet

In [23]:
from gensim.models.wrappers import LdaMallet

In [26]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = './topicmod_mallet/mallet-2.0.8/bin/mallet'
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

Mallet LDA: 20 topics, 5 topic bits, 11111 topic mask
Data loaded.
max tokens: 3161
total tokens: 24549
<10> LL/token: -9.02463
<20> LL/token: -8.64772
<30> LL/token: -8.49621
<40> LL/token: -8.41885

0	2.5	back sir droid time stay leave station side lot trouble target system lock field fear shield quick port dangerous waste 
1	2.5	good people back ai boy time stupid hard day make money clear lťs stop sell idea bag long end rain 
2	2.5	happen man stand listen bring tomorrow sleep cut damn place story idea world build bomb quiet town dream watch wait_minute 
3	2.5	good guy year love time big day feel hit break forget kind wait play give call work dead fine easy 
4	2.5	life good minute time thing leave stop kill put work understand shoot hour case save perfect fall wrong hear head 
5	2.5	forr funny lot friend son business big live forrest boat stuff hear college company tv forrest_gump world week hurt voice 
6	2.5	picture girl guess woman crazy thing boy pretty back make denham fella eat

In [27]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(19,
  [('money', 0.02430939226519337),
   ('clean', 0.02430939226519337),
   ('forget', 0.023204419889502764),
   ('girl', 0.022099447513812154),
   ('guy', 0.019889502762430938),
   ('head', 0.01878453038674033),
   ('face', 0.01878453038674033),
   ('gon_na', 0.017679558011049725),
   ('kill', 0.017679558011049725),
   ('build', 0.017679558011049725)]),
 (11,
  [('thing', 0.05311077389984825),
   ('hand', 0.03414264036418816),
   ('live', 0.027314112291350532),
   ('day', 0.024279210925644917),
   ('business', 0.02276176024279211),
   ('care', 0.021244309559939303),
   ('point', 0.0204855842185129),
   ('suppose', 0.019726858877086494),
   ('bring', 0.01896813353566009),
   ('people', 0.018209408194233688)]),
 (14,
  [('gun', 0.04118268215417107),
   ('money', 0.03484688489968321),
   ('pay', 0.022175290390707498),
   ('plank', 0.02006335797254488),
   ('sell', 0.01900739176346357),
   ('bite', 0.01900739176346357),
   ('card', 0.01795142555438226),
   ('hold', 0.01795142555438226)

## Поиск оптимального количества тем для LDA

In [28]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [29]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

Mallet LDA: 2 topics, 1 topic bits, 1 topic mask
Data loaded.
max tokens: 3161
total tokens: 24549
<10> LL/token: -8.3794
<20> LL/token: -8.23946
<30> LL/token: -8.18099
<40> LL/token: -8.14332

0	25	fuck man good give guy dude money talk shit happen make car time back call kill kid minute shut work 
1	25	thing make time run good back feel put people great move love find boy big year hear live nice turn 

<50> LL/token: -8.12539
<60> LL/token: -8.11272
<70> LL/token: -8.10416
<80> LL/token: -8.09592
<90> LL/token: -8.08224

0	25	fuck man good guy dude give money happen shit car back talk make kill kid put minute shut wrong thing 
1	25	time make thing run back find people day boy great move leave hear big year live nice turn stay good 

<100> LL/token: -8.07339
<110> LL/token: -8.06774
<120> LL/token: -8.0612
<130> LL/token: -8.06244
<140> LL/token: -8.06082

0	25	fuck man good guy dude give money happen shit make back car kill talk minute kid shut gun wrong thing 
1	25	time good make t

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

## Поиск доминирующей темы в каждом тексте

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

## Поиск наиболее представительного документа для каждой темы

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics